In [10]:



import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

pre_intervention_file = r"C:\Users\karan\Downloads\Mansi Thesis\Scaled\Input Scaled\Synthetic_Control_Pune - Original - Copy.xlsx"
post_intervention_file = r"C:\Users\karan\Downloads\Mansi Thesis\Scaled\Input Scaled\Scaled_Pune_Post_Data Og.xlsx"
output_file = r"C:\Users\karan\Downloads\Mansi Thesis\Scaled\Input Scaled\Causal_Effect_Analysis.xlsx"

# Load the data
pre_intervention_data = pd.read_excel(pre_intervention_file, sheet_name="Sheet1")
post_intervention_data = pd.read_excel(post_intervention_file, sheet_name="Sheet1")

# Define KPI columns to use
kpi_columns = [
    'Impressions', 'Reach', 'Likes', 'Love Reactions', 'Care Reactions', 
    'Haha Reactions', 'Wow Reactions', 'Sad Reactions', 'Angry Reactions', 
    'Comment Count', 'Share Count', 'Profile Clicks', 'Video Views', 
    'Hide Post', 'CTR (%)', 'Conversion', 'Conversion Rate (%)', 'CPC (Rs)', 
    'CPA (Rs)'
]

# Find common columns between datasets
common_columns = [col for col in kpi_columns if col in pre_intervention_data.columns and col in post_intervention_data.columns]

# Ensure there are common columns
if not common_columns:
    raise ValueError("No matching columns found between pre- and post-intervention datasets.")

# Align pre- and post-intervention data to use the same common columns
X_control = pre_intervention_data[common_columns].iloc[:, 1:]  # Exclude treated unit column
y_treated = pre_intervention_data[common_columns].iloc[:, 0]  # Treated unit (first column)
X_post_control = post_intervention_data[common_columns].iloc[:, 1:]  # Exclude treated unit column

# Ensure columns match in both datasets
if X_control.shape[1] != X_post_control.shape[1]:
    raise ValueError("Mismatch in the number of columns between pre- and post-intervention datasets.")

# Fit a linear regression model to calculate weights
reg = LinearRegression()
reg.fit(X_control, y_treated)
weights = reg.coef_

# Compute synthetic post-intervention outcomes
synthetic_post_intervention = X_post_control.dot(weights)

# Calculate causal effects
actual_post_intervention = post_intervention_data[common_columns[0]]  # Treated unit's actual outcomes
causal_effect = actual_post_intervention - synthetic_post_intervention

# Add results to the post-intervention data
post_intervention_data['Synthetic Outcome'] = synthetic_post_intervention
post_intervention_data['Causal Effect'] = causal_effect

# Export results
post_intervention_data.to_excel(output_file, index=False)
print(f"Causal effect analysis saved to {output_file}")


Causal effect analysis saved to C:\Users\karan\Downloads\Mansi Thesis\Scaled\Input Scaled\Causal_Effect_Analysis.xlsx
